In [1]:
import pandas as pd
import yfinance as yf
from tqdm import tqdm

In [2]:
tqdm.pandas()

In [3]:
df = pd.read_parquet("./700077347251945472_sentiment.parquet")

In [4]:
df = df[df["ticker_len"] != 0]

In [5]:
len(df)

223004

In [6]:
df

,id,type,timestamp,timestampEdited,callEndedTimestamp,isPinned,content,author,attachments,embeds,stickers,reactions,mentions,reference,ticker,ticker_len,pipe_sentiment_analysis,stocktwits
5,700077569315438604,Default,2020-04-15T20:18:19.337+00:00,None,None,False,DYNT going,{'avatarUrl': 'https://cdn.discordapp.com/avat...,[],[],[],[],[],None,[DYNT],1,"[{'label': 'POSITIVE', 'score': 0.987255573272...","[{'label': 'LABEL_0', 'score': 0.8246717453002..."
7,700077754254753832,Default,2020-04-15T20:19:03.43+00:00,2020-04-15T20:19:15.598+00:00,None,False,BBBY Scalp went in small,{'avatarUrl': 'https://cdn.discordapp.com/avat...,"[{'fileName': 'unknown.png', 'fileSizeBytes': ...",[],[],[],[],None,[BBBY],1,"[{'label': 'NEGATIVE', 'score': 0.980212867259...","[{'label': 'LABEL_1', 'score': 0.9965872764587..."
9,700077957150146620,Default,2020-04-15T20:19:51.804+00:00,None,None,False,We good @PJ Matlock,{'avatarUrl': 'https://cdn.discordapp.com/avat...,[],[],[],"[{'count': 2, 'emoji': {'id': '', 'imageUrl': ...","[{'discriminator': '0001', 'id': '332561722621...",None,[PJ],1,"[{'label': 'POSITIVE', 'score': 0.999849200248...","[{'label': 'LABEL_1', 'score': 0.9982830286026..."
10,700078273178107934,Default,2020-04-15T20:21:07.151+00:00,None,None,False,Two mask companies I have a bit of are NBY and...,{'avatarUrl': 'https://cdn.discordapp.com/avat...,[],[],[],[],[],None,"[NBY, OMI]",2,"[{'label': 'NEGATIVE', 'score': 0.999777257442...","[{'label': 'LABEL_0', 'score': 0.5855484604835..."
13,700078630469894164,Default,2020-04-15T20:22:32.336+00:00,None,None,False,Cat fight @ALGO,{'avatarUrl': 'https://cdn.discordapp.com/embe...,[],[],[],"[{'count': 3, 'emoji': {'id': '474674916575281...","[{'discriminator': '3844', 'id': '403030746526...",None,[ALGO],1,"[{'label': 'NEGATIVE', 'score': 0.815911710262...","[{'label': 'LABEL_0', 'score': 0.6145144104957..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492457,1030593781810069514,Default,2022-10-14T21:31:45.09+00:00,None,None,False,INPX fuckery all AH here,{'avatarUrl': 'https://cdn.discordapp.com/avat...,[],[],[],[],[],None,"[INPX, AH]",2,"[{'label': 'NEGATIVE', 'score': 0.996339678764...","[{'label': 'LABEL_1', 'score': 0.8721067905426..."
492458,1030594191148986429,Default,2022-10-14T21:33:22.684+00:00,None,None,False,INPX 7's up,{'avatarUrl': 'https://cdn.discordapp.com/avat...,[],[],[],"[{'count': 6, 'emoji': {'id': '', 'imageUrl': ...",[],None,[INPX],1,"[{'label': 'POSITIVE', 'score': 0.762047708034...","[{'label': 'LABEL_1', 'score': 0.9483324885368..."
492460,1030614052797419591,Default,2022-10-14T22:52:18.07+00:00,None,None,False,```fix\nTop Trending: LCID TOP ILAG TSLA NIO X...,{'avatarUrl': 'https://cdn.discordapp.com/avat...,[],[],[],[],[],None,"[LCID, TOP, ILAG, TSLA, NIO, XPEV, RIVN, KR, P...",10,"[{'label': 'NEGATIVE', 'score': 0.993953049182...","[{'label': 'LABEL_1', 'score': 0.9983188509941..."
492461,1030633910004101140,Default,2022-10-15T00:11:12.397+00:00,None,None,False,"INPX close above 6.50, watcher early AM on Mon...",{'avatarUrl': 'https://cdn.discordapp.com/avat...,[],[],[],"[{'count': 6, 'emoji': {'id': '', 'imageUrl': ...",[],None,"[INPX, AM]",2,"[{'label': 'NEGATIVE', 'score': 0.900314927101...","[{'label': 'LABEL_1', 'score': 0.9985287189483..."


In [132]:
df.reactions.iloc[-1]

array([{'count': 15, 'emoji': {'id': '', 'imageUrl': 'https://twemoji.maxcdn.com/v/latest/svg/1f44b.svg', 'isAnimated': False, 'name': '👋'}},
       {'count': 14, 'emoji': {'id': '', 'imageUrl': 'https://twemoji.maxcdn.com/v/latest/svg/1f440.svg', 'isAnimated': False, 'name': '👀'}},
       {'count': 11, 'emoji': {'id': '', 'imageUrl': 'https://twemoji.maxcdn.com/v/latest/svg/1f410.svg', 'isAnimated': False, 'name': '🐐'}},
       {'count': 3, 'emoji': {'id': '', 'imageUrl': 'https://twemoji.maxcdn.com/v/latest/svg/1f534.svg', 'isAnimated': False, 'name': '🔴'}}],
      dtype=object)

In [7]:
df = df.explode("ticker")

# Check ticker

In [8]:
def check_ticker(t):
    ticker = yf.Ticker(t)
    if (ticker.info['regularMarketPrice'] == None):
        return False
    return True

In [9]:
unique_tickers = set(list(df.ticker))

In [14]:
from multiprocessing import Pool


def worker(ticker):
    """worker function"""
    res = check_ticker(ticker)
    return (ticker, res)


with Pool(64) as p:
    r = list(tqdm(p.imap(worker, unique_tickers), total=len(unique_tickers)))

100% 10444/10444 [01:51<00:00, 93.27it/s] 


In [16]:
unique_tickers_map = {k:v for k, v in r}

In [21]:
unique_tickers_map['PINS']

True

In [22]:
df["valid_ticker"] = df.ticker.progress_apply(lambda t: unique_tickers_map[t])

100% 339093/339093 [00:00<00:00, 959436.54it/s] 


In [24]:
df = df[df["valid_ticker"]]

In [97]:
df.to_parquet("700077347251945472_a.parquet")

# Get Direction Label

In [54]:
from datetime import datetime
from datetime import timedelta

In [58]:
(datetime.strptime("2020-04-15", "%Y-%m-%d") + timedelta(days = 1)).strftime("%Y-%m-%d")

'2020-04-16'

In [63]:
def get_direction(row):
    return (row["Close"] - row["Open"]) > 0

In [124]:
import traceback
def query_direction(row):
    try:
        print(row)
        dt_start = datetime.strptime(row.timestamp[:10], "%Y-%m-%d")
        dt_end = dt_start+ timedelta(days = 1)
        print(dt_start, dt_end)
        t = yf.Ticker(row.ticker)
        data = t.history(interval='1d', start=dt_start.strftime("%Y-%m-%d"), end=dt_end.strftime("%Y-%m-%d"))
        row = data.iloc[0]
        print(row)
        return get_direction(row)
    except Exception as e:
        print(traceback.format_exc())
        return None

In [81]:
query_direction(df.iloc[300])

id                                                        700419004539469924
type                                                                 Default
timestamp                                      2020-04-16T18:55:03.838+00:00
timestampEdited                                                         None
callEndedTimestamp                                                      None
isPinned                                                               False
content                                                           THMO..6.85
author                     {'avatarUrl': 'https://cdn.discordapp.com/avat...
attachments                                                               []
embeds                                                                    []
stickers                                                                  []
reactions                                                                 []
mentions                                                                  []

False

In [112]:
df[["timestamp", "ticker"]]

,timestamp,ticker
7,2020-04-15T20:19:03.43+00:00,BBBY
10,2020-04-15T20:21:07.151+00:00,NBY
10,2020-04-15T20:21:07.151+00:00,OMI
22,2020-04-15T20:33:15.853+00:00,SDC
167,2020-04-15T22:31:14.775+00:00,PM
...,...,...
492460,2022-10-14T22:52:18.07+00:00,PG
492460,2022-10-14T22:52:18.07+00:00,ACI
492461,2022-10-15T00:11:12.397+00:00,INPX
492461,2022-10-15T00:11:12.397+00:00,AM


In [115]:
df.insert(0, 'uid', range(0, len(df)))

In [ ]:
df["diretion_today"] = df.progress_apply(query_direction, axis=1)
from multiprocessing import Pool


def worker(row):
    """worker function"""
    res = query_direction(row)
    return (row.uid, res)


with Pool(1000) as p:
    r = list(tqdm(p.imap(worker, df), total=len(df)))

  0% 94/227233 [00:06<3:43:17, 16.95it/s]

- NYC: Data doesn't exist for startDate = 1587009600, endDate = 1587096000
Traceback (most recent call last):
  File "/tmp/ipykernel_16403/2106563848.py", line 9, in query_direction
    row = data.iloc[0]
  File "/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py", line 931, in __getitem__
    return self._getitem_axis(maybe_callable, axis=axis)
  File "/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py", line 1566, in _getitem_axis
    self._validate_integer(key, axis)
  File "/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py", line 1500, in _validate_integer
    raise IndexError("single positional indexer is out-of-bounds")
IndexError: single positional indexer is out-of-bounds



  0% 249/227233 [00:16<4:15:32, 14.80it/s]

- EMP: No data found for this date range, symbol may be delisted
Traceback (most recent call last):
  File "/tmp/ipykernel_16403/2106563848.py", line 9, in query_direction
    row = data.iloc[0]
  File "/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py", line 931, in __getitem__
    return self._getitem_axis(maybe_callable, axis=axis)
  File "/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py", line 1566, in _getitem_axis
    self._validate_integer(key, axis)
  File "/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py", line 1500, in _validate_integer
    raise IndexError("single positional indexer is out-of-bounds")
IndexError: single positional indexer is out-of-bounds



  0% 342/227233 [00:21<3:59:39, 15.78it/s]

...

 17% 39366/227233 [43:59<2:45:58, 18.86it/s]

- CBL: Data doesn't exist for startDate = 1596686400, endDate = 1596772800
Traceback (most recent call last):
  File "/tmp/ipykernel_16403/2106563848.py", line 9, in query_direction
    row = data.iloc[0]
  File "/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py", line 931, in __getitem__
    return self._getitem_axis(maybe_callable, axis=axis)
  File "/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py", line 1566, in _getitem_axis
    self._validate_integer(key, axis)
  File "/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py", line 1500, in _validate_integer
    raise IndexError("single positional indexer is out-of-bounds")
IndexError: single positional indexer is out-of-bounds

- AVCTW: No data found for this date range, symbol may be delisted
Traceback (most recent call last):
  File "/tmp/ipykernel_16403/2106563848.py", line 9, in query_direction
    row = data.iloc[0]
  File "/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py", lin

 17% 39371/227233 [43:59<2:54:30, 17.94it/s]

- BTC: Data doesn't exist for startDate = 1596686400, endDate = 1596772800
Traceback (most recent call last):
  File "/tmp/ipykernel_16403/2106563848.py", line 9, in query_direction
    row = data.iloc[0]
  File "/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py", line 931, in __getitem__
    return self._getitem_axis(maybe_callable, axis=axis)
  File "/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py", line 1566, in _getitem_axis
    self._validate_integer(key, axis)
  File "/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py", line 1500, in _validate_integer
    raise IndexError("single positional indexer is out-of-bounds")
IndexError: single positional indexer is out-of-bounds



 17% 39390/227233 [44:01<3:57:28, 13.18it/s]

In [150]:
df.iloc[1001]

uid                                                                     1001
id                                                        701783278763638846
type                                                                 Default
timestamp                                       2020-04-20T13:16:12.16+00:00
timestampEdited                                                         None
callEndedTimestamp                                                      None
isPinned                                                               False
content                                                               AVCT 7
author                     {'avatarUrl': 'https://cdn.discordapp.com/avat...
attachments                                                               []
embeds                                                                    []
stickers                                                                  []
reactions                                                                 []

In [147]:
df.to_parquet("700077347251945472_label.parquet")